![Kayak](https://seekvectorlogo.com/wp-content/uploads/2018/01/kayak-vector-logo.png)

# Projet KAYAK

L’équipe marketing a besoin d’aide sur un nouveau projet. Après avoir fait quelques recherches sur les utilisateurs, l’équipe a découvert que **70% de leurs utilisateurs qui planifient un voyage aimeraient avoir plus d’informations sur la destination vers laquelle ils se dirigent.**  
  
Par conséquent, l’équipe marketing de Kayak aimerait créer une application qui recommandera où les gens devraient planifier leurs prochaines vacances. L’application doit être basée sur des données réelles concernant :

       *Temps
       *Hôtels dans la région

**Objectifs**  
    * Récupération les données des destinations  
    * Obtenir des données métérologiques de chaque destination  
    * Obtenir des informations sur les hôtels sur chaque destination  
    * Stocker toutes les informations ci-dessus dans un lac de données  

### Récupération des données  
  
Dans ce notebook je vais récupérer les données à partir d'API, permettant d'obtenir des informations sur la géolocalisation des villles.  
A partir de la géolocation, je vais pouvoir utiliser l'API permettant de connaître la météo.


#### Importation des librairies

In [1]:
import requests
import re
import pandas as pd
import plotly.express as px

# Utilisation API pour récupérer les coordonnées GPS 

**33 villes**  
Nous allons réaliser ce projet sur 33 villes de France

In [2]:
villes = ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

Pour utliser l'API qui permet de retrouver la géolocalisation : https://openweathermap.org/appid

In [3]:
r = requests.get('https://nominatim.openstreetmap.org/search?format=json&city=<la rochelle>') # utilisation de requests, test sur La Rochelle

In [4]:
r.text # l'ensemble des informations obtenues par l'API

'[{"place_id":256884609,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright","osm_type":"relation","osm_id":117858,"boundingbox":["46.1331664","46.1908971","-1.2419231","-1.111097"],"lat":"46.1591126","lon":"-1.1520434","display_name":"La Rochelle, Charente-Maritime, Nouvelle-Aquitaine, France métropolitaine, 17000, France","class":"boundary","type":"administrative","importance":0.8014837096874572,"icon":"https://nominatim.openstreetmap.org/ui/mapicons//poi_boundary_administrative.p.20.png"},{"place_id":257082119,"licence":"Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright","osm_type":"relation","osm_id":1215878,"boundingbox":["47.7388479","47.7647325","5.7060641","5.7454734"],"lat":"47.7470598","lon":"5.7321403","display_name":"La Rochelle, Vesoul, Haute-Saône, Bourgogne-Franche-Comté, France métropolitaine, 70120, France","class":"boundary","type":"administrative","importance":0.7332911295153213,"icon":"https://nominatim.openstreetma

In [5]:
r.json() #.json permet de structurer les informations

[{'place_id': 256884609,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'relation',
  'osm_id': 117858,
  'boundingbox': ['46.1331664', '46.1908971', '-1.2419231', '-1.111097'],
  'lat': '46.1591126',
  'lon': '-1.1520434',
  'display_name': 'La Rochelle, Charente-Maritime, Nouvelle-Aquitaine, France métropolitaine, 17000, France',
  'class': 'boundary',
  'type': 'administrative',
  'importance': 0.8014837096874572,
  'icon': 'https://nominatim.openstreetmap.org/ui/mapicons//poi_boundary_administrative.p.20.png'},
 {'place_id': 257082119,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'relation',
  'osm_id': 1215878,
  'boundingbox': ['47.7388479', '47.7647325', '5.7060641', '5.7454734'],
  'lat': '47.7470598',
  'lon': '5.7321403',
  'display_name': 'La Rochelle, Vesoul, Haute-Saône, Bourgogne-Franche-Comté, France métropolitaine, 70120, France',
  'class': 'boundary',
  'type'

**Remarque**  
Nous obtenons des données compris dans une liste, composée de dictionnaire et de liste. 

In [6]:
r.json()[0].keys() # Test pour retrouver l'information

dict_keys(['place_id', 'licence', 'osm_type', 'osm_id', 'boundingbox', 'lat', 'lon', 'display_name', 'class', 'type', 'importance', 'icon'])

In [7]:
df = pd.DataFrame(columns = ['id_ville', 'Ville', 'latitude', 'longitude','main_weather', 'expected_rain']) 
# Création d'un DataFrame 
# Avec les informations utiles pour le projets



In [8]:
df.head() # DataFrame vide

,id_ville,Ville,latitude,longitude,main_weather,expected_rain


In [9]:
for i in range(len(villes)): # Boucle, i prendra la veleur de 0 jusqu'à la taille de la liste villes
    r = requests.get(f'https://nominatim.openstreetmap.org/search?format=json&city=<{[villes[i]]}>') # A chaque itération de vais chercher la ville de la liste villes
    df.loc[i , 'id_ville'] = i # je donne un id à ma ville
    df.loc[i, 'Ville'] = villes[i] # Dans la colonne, je met le nom de la ville
    df.loc[i, 'latitude']= r.json()[0]['lat'] # Dans la colonne latitude, vais vais chercher l'infomation 'lat' de la variable r, qui contient le request de l'API
    df.loc[i, 'longitude'] = r.json()[0]['lon'] # Dans la colonne longitude, vais vais chercher l'infomation 'lon' de la variable r, qui contient le request de l'API


In [10]:
df.shape # On vérifie le shape de notre dataFrame

(33, 6)

In [11]:
df.head()

,id_ville,Ville,latitude,longitude,main_weather,expected_rain
0,0,Mont Saint Michel,48.6355232,-1.5102571,NaN,NaN
1,1,St Malo,48.649518,-2.0260409,NaN,NaN
2,2,Bayeux,49.2764624,-0.7024738,NaN,NaN
3,3,Le Havre,49.4938975,0.1079732,NaN,NaN
4,4,Rouen,49.4404591,1.0939658,NaN,NaN


Grâce à l'API, j'au pu remplir le DateFrame que j'ai créé, avec les données de géolocalisation des villes. Je vais maintenant pouvoir remplir le tableau avec les données métérologiques obtenues également avec une API

# Utilisation API pour récupérer les données métérologiques

Pour utliser l'API qui permet de retrouver la météo : https://openweathermap.org/api/one-call-api

In [12]:
r = requests.get('https://api.openweathermap.org/data/2.5/onecall?lat={}&lon={}&exclude=current,minutely,hourly,alert&appid=75658c1cce87474bec2f67560874df95'.format(df['latitude'][0],df['longitude'][0]))
# test d'utilisation de l'API
#L'API fonctionne avec la latitude et la longitude 

In [13]:
r.json() 

{'lat': 48.6355,
 'lon': -1.5103,
 'timezone': 'Europe/Paris',
 'timezone_offset': 7200,
 'daily': [{'dt': 1631534400,
   'sunrise': 1631511606,
   'sunset': 1631557444,
   'moonrise': 1631540460,
   'moonset': 1631569860,
   'moon_phase': 0.25,
   'temp': {'day': 297.45,
    'min': 286.01,
    'max': 298.28,
    'night': 292.53,
    'eve': 295.33,
    'morn': 286.37},
   'feels_like': {'day': 297.08,
    'night': 292.35,
    'eve': 295.07,
    'morn': 285.86},
   'pressure': 1013,
   'humidity': 44,
   'dew_point': 284.43,
   'wind_speed': 5.09,
   'wind_deg': 81,
   'wind_gust': 8.9,
   'weather': [{'id': 804,
     'main': 'Clouds',
     'description': 'overcast clouds',
     'icon': '04d'}],
   'clouds': 100,
   'pop': 0.29,
   'uvi': 3.55},
  {'dt': 1631620800,
   'sunrise': 1631598090,
   'sunset': 1631643716,
   'moonrise': 1631631180,
   'moonset': 0,
   'moon_phase': 0.27,
   'temp': {'day': 294.65,
    'min': 289.26,
    'max': 294.86,
    'night': 289.26,
    'eve': 290.42,
 

**Tests**  
Plusieurs test sont réalisés pour comprendre comment récupérer l'information

In [14]:
r.json()['daily'][1].keys()

dict_keys(['dt', 'sunrise', 'sunset', 'moonrise', 'moonset', 'moon_phase', 'temp', 'feels_like', 'pressure', 'humidity', 'dew_point', 'wind_speed', 'wind_deg', 'wind_gust', 'weather', 'clouds', 'pop', 'rain', 'uvi'])

In [15]:
type(r.json()['daily'])

list

In [16]:
type(r.json()['daily'][0])

dict

In [17]:
for i in range(len(villes)):  # Boucle, i prendra la veleur de 0 jusqu'à la taille de la liste villes
    r =  requests.get('https://api.openweathermap.org/data/2.5/onecall?lat={}&lon={}&exclude=current,minutely,hourly,alert&appid=75658c1cce87474bec2f67560874df95'.format(df['latitude'][i],df['longitude'][i]))
    r = r.json()
    
    expected_rain = 0 # On fixe la variable expected_rain à 0, à chaque itération sa valeur sera 0.
    for d in r['daily']: #Boucle for qui va itérer dans la clef 'daily'
        if 'rain' in d.keys(): # Si daily contient la clef 'rain' 
            expected_rain += d['pop'] + d['rain'] # addition du taux propable de précipitation et du volume de précipitation 
            
    weathers = pd.Series([d['weather'][0]['main'] for d in r['daily']]) #Création d'une Serie 
    # Boucle for qui va itérer dans la clef 'daily'
    # je récupère l'onformation de la clef 'weather', position 0 de la liste puis dans la clefs'main'
    main_weather = weathers.mode()[0] # On récupère le mode 
    
    df.loc[i,'main_weather'] = main_weather # Les résultats sont ajoutés au DataFrame
    df.loc[i,'expected_rain'] = expected_rain # Les résultats sont ajoutés au DataFrame

In [18]:
df.head()
    

,id_ville,Ville,latitude,longitude,main_weather,expected_rain
0,0,Mont Saint Michel,48.6355232,-1.5102571,Rain,5.3364
1,1,St Malo,48.649518,-2.0260409,Clouds,9.3711
2,2,Bayeux,49.2764624,-0.7024738,Clouds,9.1872
3,3,Le Havre,49.4938975,0.1079732,Clouds,7.58
4,4,Rouen,49.4404591,1.0939658,Clouds,6.43


In [19]:
df = df.sort_values(by=['expected_rain'], ascending=True)
display(df)

,id_ville,Ville,latitude,longitude,main_weather,expected_rain
7,7,Lille,50.6365654,3.0635282,Clouds,3.687
5,5,Paris,48.8566969,2.3514616,Clear,3.7847
6,6,Amiens,49.8941708,2.2956951,Clouds,4.5048
0,0,Mont Saint Michel,48.6355232,-1.5102571,Rain,5.3364
4,4,Rouen,49.4404591,1.0939658,Clouds,6.43
32,32,La Rochelle,46.1591126,-1.1520434,Rain,6.6393
3,3,Le Havre,49.4938975,0.1079732,Clouds,7.58
26,26,Collioure,42.52505,3.0831554,Clouds,8.4983
2,2,Bayeux,49.2764624,-0.7024738,Clouds,9.1872
1,1,St Malo,48.649518,-2.0260409,Clouds,9.3711


In [20]:
df = df.astype({'id_ville': int,
                'latitude': float,
                'longitude': float,
                'expected_rain': float}) 
# Je redéfini le type des variables

In [21]:
fig = px.scatter_mapbox(df.head(), lat="latitude", lon="longitude", zoom = 4,
                        hover_data = ['main_weather', 'expected_rain'], 
                        color_continuous_scale = 'Bluered',
                        mapbox_style="carto-positron")

fig.show('iframe') 

# Affichage des 5 villes les mieux classées en fonction du taux de précipitation

In [22]:
df.to_csv('destinations.csv', index=False) # Enregistrement du DataFrame qui regroupe l'ensemble des villes, leur position et la météo 

## Concluion 1  

Dans cette première partie, l'objectif etait d'obtenir des données par l'utilisation d'API.  
Je recherchais 2 informations (géolocalisation et la météo avec le taux de précipitation).  

**Difficultés**  
    * Comprendre comment fonctionne l'API.  
    * Aller chercher l'information dans des listes puis des dictionaires puis des listes......
  
**Prochaine étape**  
Nous avons obtenu un  DataFrame contenant les villes, leur géolocalisation et la météo. Maintenant l'objectif est de rechercher des hôtels pour les 5 premières villes.  
Pour cela, nous allons récupérer la donner par la méthode de scrapping.